# Inferential Statistics

We'll use our transformed data now to illustrate some Pythonic inferential statistics calculations.
* you will need to run the previous notebook in order to save the new csv file to import here

In [ ]:
import pandas as pd

df = pd.read_csv('/home/jovyan/bike_transformed.csv')

Let's see whether the mean value of bike rentals ('cnt') is different for whether it is a holiday or not.

In [ ]:
df.groupby('holiday')['cnt'].describe()

In [ ]:
df.boxplot(column=['cnt'], by='holiday')

In [ ]:
sample_yes = df[(df['holiday'] == 1)]
sample_no = df[(df['holiday'] == 0)]

In [ ]:
print(sample_yes.shape)
print(sample_no.shape)

Hm.... we could do this, but to do the t-test, we'd need the sample sizes to be the same, and only choosing 21 of 710 values seems somewhat dicey.

In [ ]:
df.groupby('weathersit')['cnt'].describe()

This has three values, so we'd need to do multivariate analysis.... let's come back to this.

In [ ]:
df.groupby('workingday')['cnt'].describe()

That's kinda better.... 231 values for non-working-day.

In [ ]:
df.boxplot(column=['cnt'], by='workingday')

In [ ]:
sample_yes = df[(df['workingday'] == 1)]
sample_no = df[(df['workingday'] == 0)]

In [ ]:
print(sample_yes.shape)
print(sample_no.shape)

In [ ]:
sample_yes = sample_yes.sample(231, random_state=0)

# for later: what effect would something like this have in contrast with a random sample?
# sample_yes = sample_yes.sort_values(by='cnt')[0:231]

In [ ]:
print(sample_yes.shape)
print(sample_no.shape)

We then need to test that the samples have the same variance.  We can do this with Levene's test.

In [ ]:
from scipy import stats

In [ ]:
stats.levene(sample_yes['cnt'], sample_no['cnt'])

p-value is above 5%, so we assume that we can accept the null hypothesis -- the variances are the same.
* side-note:  this is not true for all the samples of 231 points from this data subset!
* we are going to ignore that and move forward

Another assumption for independent t-test: the distribution of the residuals between the two groups is a normal distribution.

In [ ]:
from sklearn.preprocessing import scale
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
diff_res = scale(np.array(sample_yes['cnt']) - np.array(sample_no['cnt']))

In [ ]:
plt.hist(diff_res)

Looks normal, so that is good.

We can be even more specific about testing for a normal distribution by making the QQ plot.  This is available in the stats module as a 'probplot'.

In [ ]:
stats.probplot(diff_res, plot=plt, dist='norm');

Looks reasonably good.

Even one more test to confirm:  the Shapiro-Wilks test for normality. (If test is not significant, then the sample is normally distributed).

In [ ]:
stats.shapiro(diff_res)

p-value is high, so sample is normally distributed.

*Finally* we get to the t-test:

In [ ]:
stats.ttest_ind(sample_yes['cnt'], sample_no['cnt'])

Lo and behold, the p-value is high so we have to accept the null hypothesis.  The means are not statistically different.

But... as a final note, we must remember that we are comparing a subset of the yes's here.

In [ ]:
fig,ax = plt.subplots(1,2)
sample_yes['cnt'].plot(kind='box',ax=ax[0])
ax[0].set_ylim([0,9000])
sample_no['cnt'].plot(kind='box',ax=ax[1])
ax[1].set_ylim([0,9000])

Let's check for temperature:

In [ ]:
df.groupby('temp')['cnt'].describe()

Let's look at hot vs cold days:

In [ ]:
df['hot'] = df['temp'] > df['temp'].mean()

In [ ]:
df.groupby('hot')['cnt'].describe()

In [ ]:
df.boxplot(column=['cnt'], by='hot')

In [ ]:
sample_yes = df[(df['hot'] == 1)]
sample_no = df[(df['hot'] == 0)]

In [ ]:
print(sample_yes.shape)
print(sample_no.shape)

In [ ]:
sample_yes = sample_yes.sample(364, random_state=0)

In [ ]:
print(sample_yes.shape)
print(sample_no.shape)

We then need to test that the samples have the same variance.  We can do this with Levene's test.

In [ ]:
stats.levene(sample_yes['cnt'], sample_no['cnt'])

p-value is below 5%..... we must proceed cautiously.

Another assumption for independent t-test: the distribution of the residuals between the two groups is a normal distribution.

In [ ]:
diff_res = scale(np.array(sample_yes['cnt']) - np.array(sample_no['cnt']))

In [ ]:
plt.hist(diff_res)

Not quite normal...

We can be even more specific about testing for a normal distribution by making the QQ plot.  This is available in the stats module as a 'probplot'.

In [ ]:
stats.probplot(diff_res, plot=plt, dist='norm');

Looks reasonably good, but with a clearer deviation at higher quantile values.

Even one more test to confirm:  the Shapiro-Wilks test for normality. (If test is not significant, then the sample is normally distributed).

In [ ]:
stats.shapiro(diff_res)

p-value is low, so again sample is not quite normally distributed.

Let's still try the t-test:

In [ ]:
stats.ttest_ind(sample_yes['cnt'], sample_no['cnt'])

Very low p-value, which would indicate a significant difference.

We can use another Python library [researchpy](https://researchpy.readthedocs.io/en/latest/ttest_documentation.html) to do a test for when the residuals aren't normally distributed: Welch's t-test.

In [ ]:
import researchpy as rp

In [ ]:
# note the equal_variances = False
descriptives, results = rp.ttest(sample_yes['cnt'], 
                                 sample_no['cnt'], 
                                 equal_variances=False)

In [ ]:
descriptives

In [ ]:
print(results)

Here the p-value is low and we've done the proper t-test for when the two variances are not equal.

Let's go back to look at the weather.  The data documentation notes:
* weathersit
  1. Clear, Few clouds, Partly cloudy, Partly cloudy
  2. Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
  3. Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
  4. Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog

In [ ]:
df.groupby('weathersit')['cnt'].describe()

In [ ]:
df.boxplot(column=['cnt'], by='weathersit')

Here we can use ANOVA, to cover all three at once.

In [ ]:
stats.f_oneway(df.loc[df['weathersit'] == 1,['cnt']],
               df.loc[df['weathersit'] == 2,['cnt']],
               df.loc[df['weathersit'] == 3,['cnt']])

Teeny tiny p-value.  The means are definitely not the same between the three groups.

Another option (with a LOT more output info) is to use the statsmodels library.  See here for their [ANOVA](https://www.statsmodels.org/stable/examples/notebooks/generated/interactions_anova.html).

In [ ]:
from statsmodels.formula.api import ols

In [ ]:
result = ols('cnt ~ C(weathersit)', data = df).fit()

In [ ]:
result.summary()

The above was for the One-Way ANOVA.  Let's explore the Two-Way ANOVA.

We need that, for example, when looking at both 'weathersit' and 'hot'.

In [ ]:
model = ols('cnt ~ C(hot)', df).fit()

In [ ]:
model.summary()

In [ ]:
model = ols('cnt ~ C(hot) + C(weathersit)', df).fit()

In [ ]:
model.summary()

and we can use the statsmodels api to assess the anova of each category:

In [ ]:
import statsmodels.api as sm

In [ ]:
sm.stats.anova_lm(model)

In [ ]:
model = ols('cnt ~ C(hot) * C(weathersit)', df).fit()

In [ ]:
model.summary()

In [ ]:
sm.stats.anova_lm(model)

Finally, let's look at the paired t-test.

Here we don't have paired data, but let's look at another dataset (from 'trangel' stats-with-python repo) with hypothetical before/after values of blood glucose readings for 40 people with diabetes.

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/trangel/stats-with-python/master/data/BG-db.csv")

In [ ]:
df

In [ ]:
df[['BG 1','BG 2']].hist()

In [ ]:
df[['BG 1','BG 2']].describe()

Relatively small difference in the mean.... is it significant?

In [ ]:
df[['BG 1','BG 2']].boxplot()

Check assumptions:
* samples are independent and random
* distribution of the residuals should be normal
* variances between the two groups are equal

In [ ]:
stats.levene(df['BG 1'], df['BG 2'])

Just above the 5%.  We assume the variances are equal.

In [ ]:
bg_diff = scale(np.array(df['BG 1']) - np.array(df['BG 2']))

In [ ]:
plt.hist(bg_diff)

In [ ]:
stats.probplot(bg_diff, plot=plt, dist='norm');

In [ ]:
stats.shapiro(bg_diff)

In [ ]:
stats.ttest_rel(df['BG 1'], df['BG 2'])

To satisfy our qualms about normality, let's us researchpy and say that the variances are not equal and that we are using paired values.  This leads to the Wilcoxon signed-rank test.

In [ ]:
rp.ttest(df['BG 1'], df['BG 2'], paired = True, equal_variances = False)